In [1]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline
#you can also just write "pylab"

import nltk.classify.util
from astropy.table import Table, Column
from datetime import timedelta

In [2]:
data = pd.read_csv('Amazon.csv') #attach Amazon data to a var called data
print(data.shape) #print data rows and columns
data.head(5) #limit data to 5 rows including a header row 

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [3]:
#smData = data[0:1000] #make a variabe for data subset limited to first 1000 
#smData.shape #show how many rows and columns we have
#print(smData.shape) #print data rows and columns
#smData.head(5) #limit data to 5 rows including a header row 

In [4]:
# features from Amazon.csv to add to feature set

# Features pulled from review txt
data['reviewLen'] = data['Text'].str.len() #captures the amount of strings present in the review (i.e. review length)
data['hasEP'] = data['Text'].str.contains('!') #captures the presences of an exclaimation point

# 1st Attempt to add punctuation 
data['hasSemi'] = data['Text'].str.contains(';') #captures the presences of a ;
data['hasColon'] = data['Text'].str.contains(':') #captures the presences of a :
data['hasAt'] = data['Text'].str.contains('@') #captures the presences of a @
data['hasDolla'] = data['Text'].str.contains('$') #captures the presences of a $
data['hasNumb'] = data['Text'].str.contains('#') #captures the presences of a #
data['hasAnd'] = data['Text'].str.contains('&') #captures the presences of a &

# 2nd Attempt adding punctuation that needs \ before (error the first time)
data['hasQuestion'] = data['Text'].str.contains('\?') #captures the presences of a ?
data['hasAsterick'] = data['Text'].str.contains('\*') #captures the presences of a *
data['hasFparenthesis'] = data['Text'].str.contains('\(') #captures the presences of a (
data['hasBparenthesis'] = data['Text'].str.contains('\)') #captures the presences of a )

# add counts for punctuation 
data['epCount'] = data['Text'].str.count('!')
data['semiCount'] = data['Text'].str.count(':')
data['atCount'] = data['Text'].str.count('@')
data['dollaCount'] = data['Text'].str.count('$')
data['numbCount'] = data['Text'].str.count('#')
data['andCount'] = data['Text'].str.count('&')
data['questionCount'] = data['Text'].str.count('\?')
data['asterickCount'] = data['Text'].str.count('\*')
data['fParenthesisCount'] = data['Text'].str.count('\(')
data['bParenthesisCount'] = data['Text'].str.count('\)')

# add punctuation count and ratio of punctuation to words -- thank you jon!
data['punctuationCount'] = data['Text'].str.count('[.,!;:()/\?-@#$&]') #captures total common punctuation count
data['punctuationToWords'] = data['punctuationCount'] / data['reviewLen'] #captures ratio of punctuation to words

# add certain words

# features related to profile name 
data['nameLen'] = data['ProfileName'].str.len()
data['nameNum'] = data['ProfileName'].str.contains('\d')
data['nameNumCount'] = data['ProfileName'].str.count('\d')
data['nameQUOTE'] = data['ProfileName'].str.contains('"')
data['nameSPACE'] = data['ProfileName'].str.contains(' ')

# finding repeating names 
data['repeatUser'] = data.duplicated('ProfileName')
data['repeatSum'] = data.duplicated('Summary')
data['repeatText'] = data.duplicated('Text')

# 
data.tail(10)



,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,punctuationCount,punctuationToWords,nameLen,nameNum,nameNumCount,nameQUOTE,nameSPACE,repeatUser,repeatSum,repeatText
454990,320405,320406,B000E1HVR0,A3CO3BDS4PXKL5,E. Morris,0,0,5,1249171200,OMG... need I say more,...,9,0.016014,9,False,0,False,True,True,False,False
454991,104610,104611,B000KSTY86,A3DTJW2ZSDQZ86,M. B. Goetz,11,11,5,1270080000,Great coffee - Great price,...,6,0.025105,11,False,0,False,True,True,False,False
454992,282272,282273,B000FICDO8,A3B5OLV01PVP85,"D. Patel ""SpeedReader""",1,1,5,1258675200,Great taste with little effort.,...,16,0.027826,22,False,0,True,True,False,False,False
454993,419683,419684,B0029ZAOW8,A2HAVA4SPL9HSM,"Larry A. Chrispyn ""PUBLISHED POET""",0,2,1,1329696000,PRICE TOO HIGH NO EFFECT,...,7,0.030568,34,False,0,True,True,True,True,True
454994,115270,115271,B003E50WMY,A37FP7HWWNTW57,Gary,0,0,5,1314921600,My Poinion.,...,1,0.006944,4,False,0,False,False,True,False,False
454995,457613,457614,B008Z5L2MW,A2W7HYIQWAJ091,"Bonnie Pierce ""TexasBonnieBelle""",0,0,5,1349049600,Outstanding - what else?,...,6,0.024291,32,False,0,True,True,True,False,False
454996,283509,283510,B0045Z4JAI,AZ3GK5ZX3SVKT,gsue,2,2,2,1306195200,"Too weak, not enough umph!",...,5,0.024510,4,False,0,False,False,True,True,True
454997,292569,292570,B001BM68S4,A359CZWDE8MPMF,LC,0,0,5,1314576000,Good Stuff,...,15,0.053004,2,False,0,False,False,True,True,False
454998,156159,156160,B002YLG5MA,A1V8J8FKBZW68K,"Ansella Adams ""Valley Lady""",4,4,5,1275350400,Way to Start the Day!,...,6,0.022556,27,False,0,True,True,False,False,False
454999,119066,119067,B004IREFUM,A1MYYCC9BSYV5Y,Colorado Mom,0,0,5,1338422400,Delicious,...,5,0.018182,12,False,0,False,True,True,True,True


In [5]:
# Taking values and making vectors
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XhasEP = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XhasSemi = data.iloc[:, 15].values.reshape(data.shape[0], 1)
XhasColon = data.iloc[:, 16].values.reshape(data.shape[0], 1)
hasAt = data.iloc[:, 17].values.reshape(data.shape[0], 1)
hasDolla = data.iloc[:, 18].values.reshape(data.shape[0], 1)
hasNumb = data.iloc[:, 19].values.reshape(data.shape[0], 1)
hasAnd = data.iloc[:, 20].values.reshape(data.shape[0], 1)
hasQuestion = data.iloc[:, 21].values.reshape(data.shape[0], 1)
hasAsterick = data.iloc[:, 22].values.reshape(data.shape[0], 1)
hasFparenthesis = data.iloc[:, 23].values.reshape(data.shape[0], 1)
hasBparenthesis = data.iloc[:, 24].values.reshape(data.shape[0], 1)
epCount = data.iloc[:, 25].values.reshape(data.shape[0], 1)
semiCount = data.iloc[:, 26].values.reshape(data.shape[0], 1)
atCount = data.iloc[:, 27].values.reshape(data.shape[0], 1)
dollaCount = data.iloc[:, 28].values.reshape(data.shape[0], 1)
numbCount = data.iloc[:, 29].values.reshape(data.shape[0], 1)
andCount = data.iloc[:, 30].values.reshape(data.shape[0], 1)
questionCount = data.iloc[:, 31].values.reshape(data.shape[0], 1)
asterickCount = data.iloc[:, 32].values.reshape(data.shape[0], 1)
fParenthesisCount = data.iloc[:, 33].values.reshape(data.shape[0], 1)
bParenthesisCount = data.iloc[:, 34].values.reshape(data.shape[0], 1)
punctuationCount = data.iloc[:, 35].values.reshape(data.shape[0], 1)
punctuationToWords = data.iloc[:, 36].values.reshape(data.shape[0], 1)
nameLen = data.iloc[:, 37].values.reshape(data.shape[0], 1)
nameNum = data.iloc[:, 38].values.reshape(data.shape[0], 1)
nameNumCount = data.iloc[:, 39].values.reshape(data.shape[0], 1)
nameQUOTE = data.iloc[:, 40].values.reshape(data.shape[0], 1)
namespace = data.iloc[:, 41].values.reshape(data.shape[0], 1)
repeatUser = data.iloc[:, 42].values.reshape(data.shape[0], 1)
repeatSum = data.iloc[:, 43].values.reshape(data.shape[0], 1)
repeatText = data.iloc[:, 44].values.reshape(data.shape[0], 1)


Xtoadd = np.concatenate((XScore, XreviewLen, XhasEP, XhasSemi, XhasColon, hasAt, hasDolla, hasNumb, hasAnd, hasQuestion, hasAsterick, hasFparenthesis, hasBparenthesis, epCount, semiCount, atCount, dollaCount, numbCount, andCount, questionCount, asterickCount, fParenthesisCount, bParenthesisCount, punctuationCount, punctuationToWords, nameLen, nameNum, nameNumCount, nameQUOTE, namespace, repeatUser, repeatSum, repeatText), axis=1)

# data.head()


In [6]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

In [7]:
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(min_df=1, ngram_range=(3,3))
X = tv.fit_transform(data.Text)

#if you restrict it to 2 to the power of 17 features is a few extra, way to restrict number of features created 


In [8]:
# convert additional features to sparse matrix and concatenate onto the bag of words sparse matrix
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([X, XtoaddSparse])
X = csr_matrix(Xfinal)

TypeError: no supported conversion for types: (dtype('O'),)

In [9]:
# size of feature set
print(X.shape)

(455000, 10176762)


In [10]:
# define y
y = data.iloc[:, 12].values
y.shape

(455000,)

In [11]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [12]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [13]:
# MODEL: SVM, linear
# will also go through each row as many times as you want it to - default is 5, making changes to weights, trying to
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000806907378336 0.153003663004
 Accuracy Rate    0.999193092622 0.846996336996
True Positives    0.993757534011 0.441890676526
True Negatives                --  0.87904469022


(0.153003663003663,
 0.84699633699633703,
 0.44189067652643149,
 0.87904469022001219)

In [14]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- ---------------- --------------
         Error Rate 0.00121193092622 0.146637362637
      Accuracy Rate   0.998788069074 0.853362637363
     True Positives   0.995178233167  0.43669431398
     True Negatives               -- 0.886325725534


(0.14663736263736263,
 0.8533626373626374,
 0.43669431398021386,
 0.88632572553421929)

In [15]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00194662480377 0.0805054945055
 Accuracy Rate   0.998053375196  0.919494505495
True Positives   0.999354227656  0.328969721195
True Negatives               --   0.96621156902


(0.080505494505494504,
 0.91949450549450551,
 0.32896972119516338,
 0.96621156901962957)

In [16]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- ---------------- --------------
    Error Rate 0.00126530612245 0.153304029304
 Accuracy Rate   0.998734693878 0.846695970696
True Positives   0.994403306354 0.444688717897
True Negatives               --  0.87849920549


(0.15330402930402931,
 0.84669597069597069,
 0.44468871789747177,
 0.87849920548963178)

In [19]:
all_models_table()

NameError: name 'SVMerror' is not defined